<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/4_03_DETECTRON2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Spis Treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb)



# Detectron2 Faster R-CNN z FPN Resnet50

Detectron2 to system detekcji obiektów stworzony przez Facebook AI Research, posiadający implementacje najnowocześniejszych algorytmów wykrywania obiektów w tym Faster R-CNN, Mask R-CNN (state-of-the-art object detection algorithms). System został napisany w PyTorchu. Przez twórców został udostępniony jako biblioteka open-source, która może być wykorzystana do obsługi i rozwoju różnych projektów badawczych.

### Instalacja środowiska

Środowisko instalowane jest z repozytorium umieszczonym przez Facebook AI Research na [GitHub](https://github.com/facebookresearch/detectron2)

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -q -e detectron2_repo

Po instalacji Detectron2 w środowisku Colab niezbędne jest ponowne uruchomienie środwiska wykonawczego (Ctrl+M.)

Import podstawowych modułów:

In [ ]:
from detectron2 import model_zoo
import detectron2.utils.comm as comm
from detectron2.engine import DefaultPredictor, DefaultTrainer, HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader
from detectron2.structures import BoxMode
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

### Struktura repozytorium
Poniżej znajduje się drzewo katalogów Detectron2 i opis zawartości katalogów.

```
detectron2/

├─checkpoint  <- checkpointer 
├─config      <- domyślne konfiguracje
├─data        <- programy obsługi zbiorów danych i programy ładujące dane
├─engine      <- moduły trenowania i predykcji
├─evaluation  <- evaluatory dla podstawowych typów zbiorów danych
├─export      <- konwerter modeli detectron2 do caffe2 (ONNX)
├─layers      <- warstwy niestandardowe np.: Deformable convolution
├─model_zoo   <- wytreniwane modele
├─modeling   
│  ├─meta_arch <- meta architektury np. R-CNN, RetinaNet
│  ├─backbone  
│  │    ├─backbone.py   <- zawiera abstrakcyjną klasę bazową Backbone
│  │    ├─build.py      <- wywołanie funkcji konstruktora określonej w config
│  │    ├─fpn.py        <- zawiera klasę FPN 
│  │    ├─resnet.py     <- zawiera klasę ResNet 
│  ├─proposal_generator <- RPN
│  └─roi_heads <- ROI heads 
├─solver       <- moduły zawierające modyfikacje optimizera i schedulera
├─structures   <- klasy struktur np.: Boxes, Instances
└─utils        <- moduły użytkowe np.: visualizer, logger 	
```

### Omówienie detectron2


Architektura detectron2 zostanie omówiona na przykładzie Faster R-CNN z FPN Resnet50. Podstawową konfiguracją Faster R-CNN z FPN Resnet50 jest [Base-RCNN-FPN](https://github.com/facebookresearch/detectron2/blob/master/configs/Base-RCNN-FPN.yaml).
Base-RCNN-FPN określa przede wszystkim meta architekturę GeneralizedRCNN, Backbone Network, RPN i ROI Heads. 

<div align="center">
<img src="https://raw.githubusercontent.com/DarekGit/FACES_DNN/master/Figures/DetectronFasterRCNN.png" alt="detectron2">
<p>Rys. 1. Detectron2 Faster R-CNN z FPN Resnet50
</div>

Klasy realizujące architekturę przedstawioną na Rys. 1. są zapisane w poniższych lokalizacjach:

GeneralizedRCNN [meta_arch/rcnn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/meta_arch/rcnn.py)
 

- Backbone Network: FPN (Feature Pyramid Network) [backbone/fpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/backbone/fpn.py)
  - ResNet [backbone/resnet.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/backbone/resnet.py)

- RPN (Region Proposal Network) [proposal_generator/rpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn.py)
  - StandardRPNHead [proposal_generator/rpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn.py)
  - RPNOutput [proposal_generator/rpn_outputs.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn_outputs.py)
 
- ROI Heads: StandardROIHeads [roi_heads/roi_heads.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/roi_heads.py)
 - ROIPooler [poolers.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/poolers.py)
 - FastRCNNConvFCHead [roi_heads/box_heads.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/box_head.py)
 - FastRCNNOutputLayers [roi_heads/fast_rcnn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/fast_rcnn.py)
 - FastRCNNOutputs [roi_heads/fast_rcnn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/fast_rcnn.py)

## Backbone Network

Implementację Feature Pyramid Network zawiera klasa FPN zapisana w pliku [backbone/fpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/backbone/fpn.py). Wybór i konfiguracja sieci Resnet50 realizowana jest poprzez wywołanie funkcji [build_resnet_fpn_backbone](https://github.com/facebookresearch/detectron2/blob/9fe2edaf6f22bb1c2069ec12b9089d19069e5286/detectron2/modeling/backbone/fpn.py#L202) przyjmującej jako jeden z argumentów słownik [cfg](https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml) zawierający konfigurację całej architektury w tym głębokość modelu. Wybierając model Resnet50 otrzymujemy poniższa strukturę wynikową:

```
ResNet(
  (stem): BasicStem()              <- BasicStem blok 
  (res2):                          <- blok res2
    (0): BottleneckBlock()         <- (shortcut), stride=(1, 1)
    (1): BottleneckBlock()         <- stride=(1, 1)
    (2): BottleneckBlock()         <- stride=(1, 1)
  (res3):                          <- blok res3
    (0): BottleneckBlock()         <- (shortcut), stride=(2, 2)
    (1): BottleneckBlock()         <- stride=(1, 1) 
    (2): BottleneckBlock()         <- stride=(1, 1)
    (3): BottleneckBlock()         <- stride=(1, 1)
  (res4):                          <- blok res4
    (0): BottleneckBlock()         <- (shortcut), stride=(2, 2)
    (1): BottleneckBlock()         <- stride=(1, 1)
    (2): BottleneckBlock()         <- stride=(1, 1)
    (3): BottleneckBlock()         <- stride=(1, 1)
    (4): BottleneckBlock()         <- stride=(1, 1)
    (5): BottleneckBlock()         <- stride=(1, 1)
  (res5):                          <- blok res5
    (0): BottleneckBlock()         <- (shortcut), stride=(2, 2)
    (1): BottleneckBlock()         <- stride=(1, 1)
    (2): BottleneckBlock()         <- stride=(1, 1)
)
```
Struktura wszystkich bloków została zapisana w pliku [faster_rcnn_R_50_FPN.txt](faster_rcnn_R_50_FPN.txt) <br>
Reprezentuje ona model wygenerowany z konfiguracji [COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml](https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml).

- **BasicStem blok** jest realizowany przez klasę [BasicStem](https://github.com/facebookresearch/detectron2/blob/5b8f68436651c74608a3f4909cae570dbe23e51d/detectron2/modeling/backbone/resnet.py#L331). Składa się z konwolucji z kernel_size=(7, 7), stride=(2, 2), batch normalizacji. Po konwolucji wywołana jest funkcja aktywacji relu i max pooling kernel_size=3 i stride=2. Na wyjściu otrzymujemy tensor którego W, H są czterokrotnie mniejsze od rozmiaru obrazu na wejściu. 

- **BottleneckBlock** zastosowano aby zwiększyć głębokość sieci przy jednoczesnym zmniejszeniu parametrów. BottleneckBlock składa się z trzech konwolucji z kernel_size kolejno: (1, 1), (3, 3), (1, 1) ze stride=(1, 1). 
  
  Występują tu trzy tyby bloków BottleneckBlock:

  - BottleneckBlock z shortcut i stride=(1, 1)
```
      (res2): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv1): Conv2d(
            64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
```
  - BottleneckBlock z shortcut i stride=(2, 2)
```
      (res4): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
          (conv1): Conv2d(
            512, 256, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv2): Conv2d(
            256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv3): Conv2d(
            256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
        )
```
  - BottleneckBlock z stride=(1, 1) bez połączenie skrótowego
```
      BottleneckBlock(
          (conv1): Conv2d(
            512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=128, eps=1e-05)
          )
          (conv2): Conv2d(
            128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=128, eps=1e-05)
          )
          (conv3): Conv2d(
            128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=512, eps=1e-05)
          )
        )
```



Wyjścia bloków **res2**, **res3**, **res4**, **res5** używane jako wejścia połączeń bocznych FPN i przetwarzane są w ścieżce odgórnej. Pierwszym krokiem jest konwolucja Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1)) redukująca głębokość tensora wynikowego bloku res5 do 256. Tensor ten staje się pierwszą warstwą mapy obiektów używaną do dalszych transformacji. Na Rys. 1. opisany został jako feature map layer 5. 


Po zastosowaniu konwolucji wyjściowej Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1) jako **p5** jest pierwszym utworzonym argumentem wejściowym dla modułu FPN i ROI_HEADS. 
Następnie feature map layer 5 przetwarzany jest po raz kolejny. Ma miejsce unpsampling poprzez wywołanie metody [torch.nn.functional.interpolate](https://github.com/facebookresearch/detectron2/blob/9fe2edaf6f22bb1c2069ec12b9089d19069e5286/detectron2/modeling/backbone/fpn.py#L131)

```
top_down_features = F.interpolate(prev_features, scale_factor=2, mode="nearest")
lateral_features = lateral_conv(features)
prev_features = lateral_features + top_down_features
```

Wynik dodawany jest do bocznego połączenia z res4. Po zastosowaniu konwolucji wyjściowej otrzymywany jest **p4**. 
Analogicznie uzyskiwane są wyniki **p3**, **p2**.

Argument **p6** wykorzytywany jest jako wejściowy RPN generowany jest przez modół [LastLevelMaxPool](https://github.com/facebookresearch/detectron2/blob/9fe2edaf6f22bb1c2069ec12b9089d19069e5286/detectron2/modeling/backbone/fpn.py#L165), który realizuje max pooling z kernel_size=1, stride=2, padding=0 na wynikowym p5. 



## Region Proposal Network

Moduł RPN składa się z dwóch części: [StandardRPNHead](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn.py) i [RPNOutput](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn_outputs.py). 

```
RPN(
  (rpn_head): StandardRPNHead(
    (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (objectness_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
    (anchor_deltas): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
  )
  (anchor_generator): DefaultAnchorGenerator(
    (cell_anchors): BufferList()
  )
)
```

**StandardRPNHead** tworzą 3 konwolucje:
```
(conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
(objectness_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
(anchor_deltas): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
```
Moduł przetwarza kolejno argumenty wejściowe p2, p3, p4, p5, p6 i zwraca listę prawdopodobieństw istnienia obiektu (pred_objectness_logits) i pred_anchor_deltas.



**anchor_generator** 

[Base-RCNN-FPN](https://github.com/facebookresearch/detectron2/blob/master/configs/Base-RCNN-FPN.yaml)definiuje wielkość i proporcje. Kolejne elementy z ANCHOR_GENERATOR.SIZES odpowiadają kolejnym feature maps p2, p3, p4, p5, p6.

Pięć elementów listy ANCHOR_GENERATOR.SIZES odpowiada pięciu poziomom map obiektów (od P2 do P6). Na przykład P2 (stride = 4) ma jedną kotwicę o rozmiarze 32.

```
  ANCHOR_GENERATOR:
    SIZES: [[32], [64], [128], [256], [512]]  # One size for each in feature map
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]  # Three aspect ratios (same for all in feature maps)
```




## ROI Heads

IN_FEATURES: ["p2", "p3", "p4", "p5"]


```
  (roi_heads): StandardROIHeads(
    (box_pooler): ROIPooler(
      (level_poolers): ModuleList(
        (0): ROIAlign(output_size=(7, 7), spatial_scale=0.25, sampling_ratio=0, aligned=True)
        (1): ROIAlign(output_size=(7, 7), spatial_scale=0.125, sampling_ratio=0, aligned=True)
        (2): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0, aligned=True)
        (3): ROIAlign(output_size=(7, 7), spatial_scale=0.03125, sampling_ratio=0, aligned=True)
      )
    )
    (box_head): FastRCNNConvFCHead(
      (fc1): Linear(in_features=12544, out_features=1024, bias=True)
      (fc2): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (box_predictor): FastRCNNOutputLayers(
      (cls_score): Linear(in_features=1024, out_features=2, bias=True)
      (bbox_pred): Linear(in_features=1024, out_features=4, bias=True)
    )
  )
```